# Search Engine Experiments

Test of different tokenization methods and Word Embedding for the search engine of the articles of Equinox by Asesoftware

## CSV of Articles

CSV Columns: “article_name”, “enumeration_in_article”, “content” 
“stringWithAllText”

In [1]:
import pandas as pd
df = pd.read_csv("articles_paragraphs.csv")

In [2]:
df_eng = df[df['language_code'] =='en'].reset_index()

In [3]:
df_eng

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code
0,0,0,Enhance medicine using AI,"During the last decades, AI transformed multip...",0,0,en
1,1,1,Enhance medicine using AI,The first time AI helped humans research was i...,1,0,en
2,2,2,Enhance medicine using AI,AI is turning the drug-discovery paradigm upsi...,2,0,en
3,3,3,Enhance medicine using AI,Another contribution of AI to this field was m...,3,0,en
4,4,4,Enhance medicine using AI,Then this information is used to create a repr...,4,0,en
...,...,...,...,...,...,...,...
383,537,537,BI for making decisions,"Data Analysis Reporting: In this step, data vi...",11,20,en
384,538,538,BI for making decisions,Decision Making: This last step is where the r...,12,20,en
385,539,539,BI for making decisions,BI can have different applications for one org...,13,20,en
386,540,540,BI for making decisions,"In conclusion, BI is important for organizatio...",14,20,en


## Data Preprocessing and Tokenization

### Lemma Whitespace Tokenization

In [4]:
import pandas as pd
import string
import spacy

'''
In this example, we use the Spacy library to preprocess and tokenize the text, 
lowercasing the text, removing punctuation, lemmatizing the words, and removing stopwords 
and short words. We then apply this function to each paragraph in the 'content' column of the CSV file using a for loop, 
and append the resulting list of tokens to a list of lists. The final result is a list of lists, where each 
sublist contains the tokens of each paragraph.

'''

# load spacy nlp model
nlp = spacy.load('en_core_web_sm')

# define function for pre-processing and tokenization
def preprocess_text_lemma(text):
    # lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # lemmatize
    doc = nlp(text)
    lemmatized_text = [token.lemma_ for token in doc]
    # remove stopwords and short words
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    tokens = [token for token in lemmatized_text if token not in stopwords and len(token) > 2]
    return tokens

# apply pre-processing and tokenization to the 'content' column of each row
tokenized_paragraphs_lemma_eng = []
for paragraph in df_eng['content']:
    tokens = preprocess_text_lemma(paragraph)
    tokenized_paragraphs_lemma_eng.append(tokens)

# print the resulting list of lists of tokens
print(tokenized_paragraphs_lemma_eng)


2023-04-14 15:15:26.776853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 15:15:26.968989: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-14 15:15:26.983689: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 15:15:26.983934: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

[['decade', 'transform', 'multiple', 'field', 'knowledge', 'medicine', 'transformation', 'different', 'way', 'enhance', 'medicine', 'use', 'article', 'introduce', 'help', 'discover', 'new', 'drug', 'understand', 'mystery', 'cancer', 'learn', 'billion', 'relation', 'different', 'research', 'resource'], ['time', 'help', 'human', 'research', '2007', 'adam', 'robot', 'generate', 'hypothesis', 'gene', 'code', 'critical', 'enzyme', 'catalyze', 'reaction', 'yeast', 'saccharomyce', 'cerevisiae', 'adam', 'use', 'robotic', 'test', 'prediction', 'lab', 'physically', 'researcher', 'university', 'aberystwyth', 'cambridge', 'independently', 'test', 'adamsadam', 'hypothese', 'function', 'gene', 'new', 'accurate', 'wrong', 'example', 'multiple', 'application', 'field', 'ready', 'learn'], ['turn', 'drugdiscovery', 'paradigm', 'upside', 'use', 'patientdriven', 'biology', 'datum', 'derive', 'morepredictive', 'hypothesis', 'traditional', 'trialanderror', 'approach', 'example', 'boston', 'berg', 'biotechno

## Word Embedding

### Lemma Whitespace

In [5]:
import gensim
import numpy as np

'''
Here we train the Word2Vec model with a list of lists where each sublist is a tokenized paragraph.
After we get the word vectors per paragraph, we compute our paragraph meaning vector as the mean
of its word vectors.
'''

# Train Word2Vec model
lemmaModel = gensim.models.Word2Vec(tokenized_paragraphs_lemma_eng, window=20, min_count=1, workers=4)
lemmaModel.save("paragraphModel")

# Calculate the meaning vector per paragraph
paragraph_vectors_lemma = []
for paragraph_tokens in tokenized_paragraphs_lemma_eng:
    vectors = []
    for token in paragraph_tokens:
        if token in lemmaModel.wv.key_to_index:
            vectors.append(lemmaModel.wv[token])
    if len(vectors) > 0:
        paragraph_vectors_lemma.append(np.mean(vectors, axis=0))
    else:
        paragraph_vectors_lemma.append(np.zeros(lemmaModel.vector_size))

print(paragraph_vectors_lemma[383])

[-0.03620399  0.02469524  0.00266066  0.00516001 -0.00311047 -0.07632554
  0.01775371  0.07823075 -0.01582125 -0.00646961 -0.02242069 -0.04970263
 -0.0205809   0.00830256  0.0078301  -0.02338822 -0.00337319 -0.04164539
 -0.00379483 -0.06241579  0.01994985  0.0090849   0.01259553 -0.01413945
 -0.01432433  0.01239184 -0.03662726 -0.02438568 -0.02918436  0.0080363
  0.03996354  0.01687322  0.00963097 -0.00110473 -0.02482542  0.04977199
 -0.00356381 -0.03516663 -0.01884404 -0.06706529 -0.00583096 -0.02255758
 -0.014777    0.00224524  0.02197856 -0.01618789 -0.02491317 -0.00419469
  0.03002756  0.01782669  0.01487968 -0.03316006 -0.01417048  0.01490296
 -0.031634    0.02251959  0.03341629 -0.00551052 -0.02941393  0.01093525
  0.00403567  0.01291608 -0.01657325 -0.00010213 -0.03250395  0.02462719
  0.00539508  0.02101104 -0.04002587  0.04262671 -0.02847364  0.0173063
  0.04233714 -0.0228856   0.02943371  0.01468018 -0.00902918 -0.01049583
 -0.03992293  0.00779966 -0.0206516   0.00228876 -0.0

#### Agregamos los vectores con su respectivo tokenization


In [6]:
df_eng['vector'] = paragraph_vectors_lemma


### PoS tagging

In [7]:
import nltk
from gensim.models import Word2Vec
import numpy as np

Creamos las tuplas con sus tags

In [8]:
nltk.download('averaged_perceptron_tagger')

tagged_sentences_eng = [nltk.pos_tag((paragraph)) for paragraph in tokenized_paragraphs_lemma_eng]

# Entrenamiento del modelo Word2Vec
model_eng = Word2Vec(tagged_sentences_eng,  window=20, min_count=5, workers=4)

# Utilizar el modelo Word2Vec para encontrar palabras similares



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/felipe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Creamos los vectores por cada parrafo

In [9]:
# Calculate the meaning vector per paragraph
paragraph_tag_vectors_lemma_eng = []
for paragraph_tokens in tagged_sentences_eng:
    vectors = []
    ind = 0
    for ind, token in enumerate(paragraph_tokens):
        ind = ind + 1
        if ind in range(len(tagged_sentences_eng)):
            vectors.append(model_eng.wv[ind])
    if len(vectors) > 0:
        paragraph_tag_vectors_lemma_eng.append(np.mean(vectors, axis=0))
    else:
        paragraph_tag_vectors_lemma_eng.append(np.zeros(model_eng.vector_size))

In [10]:
df_eng['vector_tag'] = paragraph_tag_vectors_lemma_eng

In [11]:
df_eng.head()

,index,Unnamed: 0,article_name,content,enumeration_in_article,file_id,language_code,vector,vector_tag
0,0,0,Enhance medicine using AI,"During the last decades, AI transformed multip...",0,0,en,"[-0.032288928, 0.021287974, -0.00051966205, 0....","[-0.048813313, 0.004975025, 0.077069245, 0.035..."
1,1,1,Enhance medicine using AI,The first time AI helped humans research was i...,1,0,en,"[-0.023940086, 0.01433464, 0.0009171194, 0.004...","[-0.045571435, 0.0030061977, 0.07079606, 0.031..."
2,2,2,Enhance medicine using AI,AI is turning the drug-discovery paradigm upsi...,2,0,en,"[-0.025652029, 0.016781839, 0.000320767, 0.003...","[-0.04777068, 0.003595353, 0.0752004, 0.034602..."
3,3,3,Enhance medicine using AI,Another contribution of AI to this field was m...,3,0,en,"[-0.027229961, 0.016531883, -0.00047050405, 0....","[-0.054185208, 0.0060726693, 0.08301202, 0.036..."
4,4,4,Enhance medicine using AI,Then this information is used to create a repr...,4,0,en,"[-0.026715368, 0.014124828, 0.0014429186, 0.00...","[-0.048600964, 0.0049037286, 0.07690233, 0.035..."


## Similarity Function

In [12]:
import numpy as np
from gensim.models import KeyedVectors


def cosine_similarity_list(vectors_list, query_vector):
    #Compute the cosine similarity between the vector representation of the input and the vector representations of each sentence in the text
    similarity_scores = []
    for vector in vectors_list:
        score = query_vector.dot(vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
        similarity_scores.append(score)

    # Sort the sentences in descending order of their cosine similarity to the input and return the top-N most similar sentences
    n = 100
    most_similar_sentences = [[vectors_list[idx],idx] for idx in np.argsort(similarity_scores)[::-1][:n] if np.sum(vectors_list[idx]) != 0]

    return most_similar_sentences[:20]


In [13]:
cosine_similarity_list(df_eng['vector'],df_eng['vector'][0])

[[array([-0.03228893,  0.02128797, -0.00051966,  0.00490977, -0.00163631,
         -0.06383732,  0.01608596,  0.06375008, -0.01232022, -0.00351368,
         -0.01815723, -0.04110382, -0.01233967,  0.00760725,  0.00793665,
         -0.02263543, -0.00355799, -0.03593039, -0.00179565, -0.05428541,
          0.01740661,  0.00887995,  0.01063564, -0.01212995, -0.01068683,
          0.00767084, -0.02967313, -0.02088034, -0.02614477,  0.00637032,
          0.03153363,  0.01239618,  0.00841742,  0.00054091, -0.019744  ,
          0.04158318, -0.00380785, -0.0294814 , -0.01667364, -0.05277834,
         -0.00550628, -0.01606686, -0.01280814,  0.00274895,  0.01928761,
         -0.01416461, -0.01651307, -0.0043103 ,  0.02785651,  0.01861892,
          0.01255749, -0.02752165, -0.01305897,  0.0114572 , -0.02749982,
          0.01601272,  0.02802273, -0.00442311, -0.02561128,  0.00973356,
          0.00346251,  0.01337022, -0.01220104, -0.00269802, -0.02942267,
          0.01817472,  0.00412611,  0.

## Prompt Preprocessing, Tokenization and Embedding

### Lemma with Pos-tagging

In [18]:
userPrompt="nlp"
tokenized_prompt = preprocess_text_lemma(userPrompt)
tagged_prompt = nltk.pos_tag(tokenized_prompt)
print(tagged_prompt)
promptVector_lemma_eng = np.zeros((model_eng.vector_size,))
word_count  = 0
ind = 0
for ind,token in enumerate(tokenized_prompt):
    ind = ind + 1
    if token in tokenized_prompt:
        promptVector_lemma_eng += model_eng.wv[ind]
        word_count += 1
        print(word_count,ind)

if word_count > 0:
    promptVector_lemma_eng /= word_count
    





[('nlp', 'NN')]
1 1


### Testing PoS tagging

In [19]:
var = cosine_similarity_list(df_eng['vector_tag'],promptVector_lemma_eng)

## Similarity Test

In [20]:
var=np.array(var)
var[:,1]
possible_solutions=df_eng.iloc[var[:,1]]

/tmp/ipykernel_3276/52625493.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  var=np.array(var)


In [21]:
for paragraph in possible_solutions["content"]:
    print(paragraph, "\n")

And what about these? Perhaps the sensations are better than the previous ones. 

BI can have different applications for one organization; among them, we can find [2]: 

If you want to read more of our content follow the link : 

Despite their differences, natural thinking and AI can complement each other. 

For example, if the username is myjetson and IP is 192.168.101.15, then the command will be: 

Rankings-Creepiest-Robots - ROBOTS: Your Guide to the World of Robotics.. Robots.Ieee. 

Haboo. (s. f.). [Avatars at the disco in Haboo]. Haboo. https://www.habbo.es/playing-habbo/what-is-habbo 

On VNC Viewer, go to File > New connection > General. 

This output shows the different internet connections on the Jetson Nano where: 

- Language detection: you want to determine a given text's language. 

If you want to know more about Quantum Computing read my blog 

The RPA solutions will be a long-term solution (It will be executed several times) 

It is a rule-based process (It follows a w